In [46]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from multiprocessing import Pool
import tensorflow as tf
from tensorflow.python.ops import nn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda, Dropout
from tensorflow.keras import Input, Model
from utils import *
from functions import * 
from DNN_metrics import * 
from loss_function import * 
from model_DL import *
from metrics import *
py_file_location = "..."
os.path.abspath(os.path.join(os.path.dirname(py_file_location), os.path.pardir))





tf.device('GPU:1') 

## Loading the data

In [47]:
Nbr_train = int(1E6)

Nbr_test = int(2E5) 


#outfile = 'Dataset/' # choose your path 

In [48]:


project_sub_path = "dataset"
  
# Parent Directory path
parent_dir = ""
### Train ###
#dataset_train_Anne
dataset_train = np.load(os.path.join(parent_dir,project_sub_path,'dataset_train_GF.npz'))

h_11_tr = dataset_train['h_PP'][:Nbr_train]
h_12_tr = dataset_train['h_PS'][:Nbr_train]
h_1R_tr = dataset_train['h_PR'][:Nbr_train]
h_21_tr = dataset_train['h_SP'][:Nbr_train]
h_22_tr = dataset_train['h_SS'][:Nbr_train]
h_2R_tr = dataset_train['h_SR'][:Nbr_train]
h_R1_tr = dataset_train['h_RP'][:Nbr_train]
h_R2_tr = dataset_train['h_RS'][:Nbr_train]


#x_train_input = np.stack([h_11_tr, h_12_tr, h_1R_tr, h_21_tr, h_22_tr, h_2R_tr, h_R1_tr, h_R2_tr], axis=1)
x_train = np.stack([h_R1_tr, h_11_tr, h_2R_tr, h_1R_tr, h_22_tr, h_R2_tr, h_21_tr, h_12_tr], axis=1)





### VAL ###
  
# Parent Directory path
#dataset_val_Anne

dataset_test = np.load(os.path.join(parent_dir,project_sub_path,'dataset_val_GF.npz'))

h_11_test = dataset_test['h_PP'][:Nbr_test]
h_12_test = dataset_test['h_PS'][:Nbr_test]
h_1R_test = dataset_test['h_PR'][:Nbr_test]
h_21_test = dataset_test['h_SP'][:Nbr_test]
h_22_test = dataset_test['h_SS'][:Nbr_test]
h_2R_test = dataset_test['h_SR'][:Nbr_test]
h_R1_test = dataset_test['h_RP'][:Nbr_test]
h_R2_test = dataset_test['h_RS'][:Nbr_test]

x_test = np.stack([h_R1_test, h_11_test, h_2R_test, h_1R_test, h_22_test, h_R2_test, h_21_test, h_12_test], axis=1)



# Performance metrics

In [49]:
Nbr_train = int(1E6)

 
tau = 0.25

VS = 0.2 # validation_split

Epochs = 1 # Epochs number

BS = 4096 # batch_size

LD = 10**0.5

LR = 10**-4

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

metrics = [Delta_DNN_CF, Opportunistic_Achievable_Rate_CF(tau),outage_percentage_CF(tau)] #, QoS_mean_DF, QoS_median_DF



#Nbr_train = int(1E6)

model = get_model_CF(x_train, loss_CF(LD,tau), metrics, custom_sigmoid, custom_sigmoid, LR) #lr_v
history = model.fit(x_train, np.power(x_train,2), epochs=Epochs, batch_size=BS, validation_split = VS,callbacks=[callback])

model.save('WM_CF/CF.h5')
np.save('WM_CF/CF',history.history)




Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 128)          1152        input_4[0][0]                    
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 256)          33024       dense_18[0][0]                   
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 256)          65792       dense_19[0][0]                   
____________________________________________________________________________________________

# Part 2 

# Part 3 channel noise

## Analytic solution to noisy test set and test set  without noise ...

In [7]:
P1 = 10.0
tau = 0.25
Primary_ID = [0, 1, 6] 
Secondary_ID = [3, 7]



In [8]:
# # choose your directory path  
project_sub_path = "Imperfect CSI Data CF/"
  
# Parent Directory path
parent_dir = ""
  
# Path
try : 

    path = os.path.join(parent_dir, project_sub_path)
    os.mkdir(path)
    print("Directory '% s' created" % project_sub_path)
except FileExistsError : 
    pass
    



In [9]:
# Get Noisy H-MATRIX with different noise variance
try : 
    noise_levels, NH_MATRIX = noise_to_channels(x_test, Primary_ID, Secondary_ID)



    # Compute bruteforce for NH-MATRIX

    AS_NG_MATRIX = AS_for_noisy_channels(NH_MATRIX) 

    # bruteforce for test_set without noise

    AS_G_Benchmark = generate_benchmark(x_test) 

    #----------------------------------Save data----------------------------------------------#

    outfile_NH_MATRIX = project_sub_path+'/NH_MATRIX_GF_CDIT'
    np.savez(outfile_NH_MATRIX, NH_MATRIX)


    outfile_BF_NG_MATRIX = project_sub_path+'/BF_NG_MATRIX_GF_CDIT'
    np.savez(outfile_BF_NG_MATRIX, AS_NG_MATRIX)


    outfile_BF_G_Benchmark = project_sub_path+'/BF_G_Benchmark_GF_CDIT'
    np.savez(outfile_BF_G_Benchmark, AS_G_Benchmark)

except FileExistsError : 
    pass

## loading the data 

In [10]:
# First, load ... 
# useful for DNN prediction because DNN takes H as input 

outfile_NH_MATRIX = project_sub_path+'/NH_MATRIX_GF_CDIT'

NH_MATRIX = np.load(outfile_NH_MATRIX+".npz")
NH_MATRIX.files
NH_MATRIX = NH_MATRIX['arr_0']

# Noisy_H ==> NH_MATRIX

#-----------------------------------------------------------------------------------------#
# useful for Rate, outage, Delta calculation ...  because it containing (Alpha, Pr, Ps) 


outfile_BF_NG_MATRIX = project_sub_path+'/BF_NG_MATRIX_GF_CDIT'

dataset_test = np.load(outfile_BF_NG_MATRIX+".npz")
dataset_test.files
dataset_test = dataset_test['arr_0']

#-----------------------------------------------------------------------------------------#
# useful for Rate, outage, Delta calculation, because it containing G Matrix without noise 


outfile_BF_G_Benchmark = project_sub_path+'/BF_G_Benchmark_GF_CDIT'

dataset = np.load(outfile_BF_G_Benchmark+".npz")
dataset.files
dataset = dataset['arr_0']






## test Analytic solution with noisy channels

In [11]:
db_bruteforce = opportunistic_rate_for_noisy_channels_CF(dataset, dataset_test) 
# rate for H without noise
db_benchmark = C(f_obj(dataset[:,0], dataset[:,1], dataset[:,2], dataset[:,3], dataset[:,4], dataset[:,5], dataset[:,6], dataset[:,7], dataset[:,8], dataset[:,9]))


# List of relative gap between each noisy matrix and benchmark (BF results without noise)

db_gap = np.array([relative_avreage_gap(db_bruteforce[0,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[1,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[2,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[3,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[4,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[5,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[6,:], db_benchmark[:])])

db_gap = db_gap*100



SNRs_db = [-10, -5, 0, 5, 10, 15, 20]



In [13]:




Mean_ARPD, Max_ARPD, Mean_Outage_ARPD, Outage_ARPD = PARD_for_noisy_data_CF(dataset,dataset_test)



## Deep Neural Network (trained without noise)

## test DNN with noisy channels

In [ ]:
#"weights_model/gaussian_fading_model/anne_model.h5"
Model_Weight_Path = "WM_CF/CF.h5"



DNN_pred = DNN_predictions_CF(NH_MATRIX, Model_Weight_Path)
db_DNN = opportunistic_rate_for_noisy_channels_DNN_CF(dataset, DNN_pred)
db_benchmark = C(f_obj(dataset[:,0], dataset[:,1], dataset[:,2], dataset[:,3], dataset[:,4], dataset[:,5], dataset[:,6], dataset[:,7], dataset[:,8], dataset[:,9]))


db_gap_DNN = np.array([relative_avreage_gap(db_DNN[0,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[1,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[2,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[3,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[4,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[5,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[6,:], db_benchmark[:])])
#,\
#                       relative_avreage_gap(db_DNN[7,:,8], db_benchmark[:,8])


#ax.plot(variance, db_gap, c = 'blue',label='BF')

db_gap_DNN = db_gap_DNN*100



In [26]:
Mean_ARPD, Max_ARPD, Mean_Outage_ARPD, Outage_ARPD = PARD_for_noisy_data_DNN_CF(dataset, DNN_pred)


## Training DNN with noisy channels

## Noise to train set

In [30]:

# Get P and S channels index form dataset (h_R1, h_11, h_2R, h_1R, h_22, h_R2, h_21, h_12)

Primary_ID = [0, 1, 6] 
Secondary_ID = [3, 7]

try : 

    noise_levels, X_train_noised = noise_to_channels(x_train, Primary_ID, Secondary_ID)
    X_train_noised_path = project_sub_path+'X_train_noised_GF_CDIT'
    np.savez(X_train_noised_path, X_train_noised)

except FileExistsError : 
    pass





In [31]:
#X_train_noised_GF = 'Dataset_VF/X_train_noised_anne'
X_train_noised_path = project_sub_path+'X_train_noised_GF_CDIT'

X_train_noised = np.load(X_train_noised_path+".npz")
X_train_noised.files
X_train_noised = X_train_noised['arr_0']



In [32]:

#-----------------------------------------------------------------------------------------#        
Nbr_train = int(1E6)

 
tau = 0.25

VS = 0.2 # validation_split

Epochs = 1 # Epochs number

BS = 4096 # batch_size

LD = 10**0.5

LR = 10**-4


metrics = [Delta_DNN_CF, Opportunistic_Achievable_Rate_CF(tau),outage_percentage_CF(tau)] #, QoS_mean_DF, QoS_median_DF



#Nbr_train = int(1E6)

hyperparameters = {'lambda' : LD, 'tau': tau, 'metrics': metrics, 'learning_rate': LR, 'epochs':Epochs, 'batch_size': BS, 'validation_split': VS}
#hyperparameters = [LD, tau, metrics, LR, Epochs, BS, VS]
DNN_for_noisy_channels_CF(x_train[:Nbr_train,:], X_train_noised[:,:Nbr_train,:], hyperparameters)






Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 128)          1152        input_3[0][0]                    
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 256)          33024       dense_12[0][0]                   
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 256)          65792       dense_13[0][0]                   
____________________________________________________________________________________________

KeyboardInterrupt: 

In [36]:
        


DNN_pred = noised_DNN_pred_CF(NH_MATRIX, 'Weights_Model/CSI imperfection/CDIT/v2_')



In [37]:

loss_history, val_loss_history = plot_history_noised_DNN('Weights_Model/CSI imperfection/CDIT/v2_')    



In [42]:
db_DNN = opportunistic_rate_for_noisy_channels_DNN(dataset, DNN_pred)
db_benchmark = C(f_obj(dataset[:,0], dataset[:,1], dataset[:,2], dataset[:,3], dataset[:,4], dataset[:,5], dataset[:,6], dataset[:,7], dataset[:,8], dataset[:,9]))


Mean_ARPD_DNN, Max_ARPD_DNN, Mean_Outage_ARPD_DNN, Outage_ARPD_DNN = PARD_for_noisy_data_DNN(dataset, DNN_pred)



In [44]:
X = {"opportunistic_rate":db_DNN, "mean_ARPD":Mean_ARPD_DNN,"max_ARPD":Max_ARPD_DNN,\
    "mean_outage":Mean_Outage_ARPD_DNN, "outage_ARPD":Outage_ARPD_DNN}

rate_gap_dnn, mean_pard_dnn, max_pard_dnn, mean_outage_pard_dnn, outage_pard_dnn = get_dnn_results(X, db_benchmark)